In [1]:
import pyterrier as pt
import os
import numpy as np
import pandas as pd
import fastrank
import requests
import datetime

import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor

In [2]:
if not pt.started():
    pt.init()

PyTerrier 0.9.1 has loaded Terrier 5.7 (built by craigm on 2022-11-10 18:30) and terrier-helper 0.0.7

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


## Downloading the trec covid dataset

In [3]:
dataset = pt.datasets.get_dataset('irds:cord19/trec-covid')
pt_index_path = './indices/cord19'

if not os.path.exists(pt_index_path + "/data.properties"):
    indexer = pt.index.IterDictIndexer(pt_index_path, blocks=True)
    index_ref = indexer.index(dataset.get_corpus_iter(), 
                            fields=['title', 'doi', 'abstract'], 
                            meta=('docno',))
else:
    index_ref = pt.IndexRef.of(pt_index_path + "/data.properties")

In [4]:
metadata = pd.read_csv('~/.ir_datasets/cord19/2020-07-16/metadata.csv')

/Users/nicolasrehbach/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (1,4,5,6,13,14,15,16) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
index = pt.IndexFactory.of(index_ref)
topics = dataset.get_topics('title')
qrels = dataset.get_qrels()

In [6]:
qrels

,qid,docno,label,iteration
0,1,005b2j4b,2,4.5
1,1,00fmeepz,1,4
2,1,010vptx3,2,0.5
3,1,0194oljo,1,2.5
4,1,021q9884,1,4
...,...,...,...,...
69313,50,zvop8bxh,2,5
69314,50,zwf26o63,1,5
69315,50,zwsvlnwe,0,5
69316,50,zxr01yln,1,5


In [7]:
len(metadata['journal'].unique())

18112

### How do pipes work

We set up a two-stage ranking pipeline with a BM25 first stage ranker whose outputs will be reranked by a ML method given the predefined featues.

In [8]:
BM25 = pt.BatchRetrieve(index, controls={"wmodel": "BM25"})
TF_IDF = pt.BatchRetrieve(index, controls = {"wmodel":"TF_IDF"})
PL2 = pt.BatchRetrieve(index, controls = {"wmodel": "PL2"})

We make a pipe by transforming the BM25 outputs with the help of PL2 and TFIDF

In [9]:
pipe = BM25 >>(TF_IDF ** PL2)

Alternative: FeatureBatchRetrieve object

In [10]:
fbr = pt.FeaturesBatchRetrieve(index, controls = {"wmodel": "BM25"}, features=["WMODEL:TF_IDF", "WMODEL:PL2"]) 
(fbr % 5).search("coronavirus immunity")

,qid,query,docid,rank,features,docno,score
0,1,coronavirus immunity,187945,0,"[4.540221677858543, 3.477440707307063]",sp212tai,10.118259
1,1,coronavirus immunity,126990,1,"[4.377525236902022, 3.3080236155861185]",e1mw9lx1,10.001470
2,1,coronavirus immunity,179948,2,"[4.559628851404359, 3.5124279884117215]",ltmuw6f8,9.974369
3,1,coronavirus immunity,156456,3,"[4.600248221319108, 3.649953983552013]",1oruu33o,9.955978
4,1,coronavirus immunity,94922,4,"[4.490926556339275, 3.312256733881456]",5jl6ltfj,9.734640


### LTR

Split into train test using cross validation

In [11]:
train_topics, validation_topics, test_topics = np.split(topics, [int(.6*len(topics)), int(.8*len(topics))])

In [12]:
train_min = train_topics['qid'].astype(int).min()
train_max = train_topics['qid'].astype(int).max()
train_qrels = qrels[(qrels['qid'].astype(int) >= train_min) & (qrels['qid'].astype(int) <= train_max)]

val_min = validation_topics['qid'].astype(int).min()
val_max = validation_topics['qid'].astype(int).max()
validation_qrels = qrels[(qrels['qid'].astype(int) >= val_min) & (qrels['qid'].astype(int) <= val_max)]

test_min = test_topics['qid'].astype(int).min()
test_max = test_topics['qid'].astype(int).max()
test_qrels = qrels[(qrels['qid'].astype(int) >= test_min) & (qrels['qid'].astype(int) <= test_max)]

In [13]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators=200)
rf_pipe = fbr >> pt.ltr.apply_learned_model(rf)
rf_pipe.fit(train_topics, qrels)

In [14]:
results = pt.Experiment([PL2, rf_pipe], test_topics, qrels, ["map"], names = ["PL2(Baseline)", "Random Forest"])

In [15]:
results

,name,map
0,PL2(Baseline),0.313517
1,Random Forest,0.141776


### Using Gradient Boosted Trees and LambdaMART

In [16]:
import xgboost as xgb

lmart_x = xgb.sklearn.XGBRanker(objective='rank:ndcg',
      learning_rate=0.1,
      gamma=1.0,
      min_child_weight=0.1,
      max_depth=6,
      verbose=2,
      random_state=42)

lmart_x_pipe = fbr >> pt.ltr.apply_learned_model(lmart_x, form="ltr")
lmart_x_pipe.fit(train_topics, train_qrels, validation_topics, validation_qrels)

[16:57:20] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/learner.cc:767: 
Parameters: { "verbose" } are not used.



In [17]:
train_request = fastrank.TrainRequest.coordinate_ascent()
params = train_request.params
params.init_random = True
params.normalize = True
params.seed = 1234567

ca_pipe = fbr >> pt.ltr.apply_learned_model(train_request, form="fastrank")
ca_pipe.fit(train_topics, qrels)

---------------------------
Training starts...
---------------------------
[+] Random restart #1/5...
[+] Random restart #2/5...
[+] Random restart #3/5...
[+] Random restart #5/5...
[+] Random restart #4/5...
Shuffle features and optimize!
----------------------------------------
   0|Feature         |   Weight|     NDCG
----------------------------------------
Shuffle features and optimize!
----------------------------------------
   4|Feature         |   Weight|     NDCG
----------------------------------------
Shuffle features and optimize!
----------------------------------------
   3|Feature         |   Weight|     NDCG
----------------------------------------
   3|1               |    0.000|    0.696
Shuffle features and optimize!
----------------------------------------
   2|Feature         |   Weight|     NDCG
----------------------------------------
Shuffle features and optimize!
----------------------------------------
   1|Feature         |   Weight|     NDCG
--------------

In [18]:
pt.Experiment(
    [PL2, lmart_x_pipe, ca_pipe],
    test_topics,
    test_qrels,
    ["map"],
    names=["PL2 Baseline", "LambdaMART (xgBoost)", 'FastRank'], filter_by_qrels = True 
)

,name,map
0,PL2 Baseline,0.313517
1,LambdaMART (xgBoost),0.167785
2,FastRank,0.236252


### Applying custom features

In [19]:
# for faster access write the author information into a dictionary
author_dict = {}
for id, authors in zip(metadata['cord_uid'], metadata['authors']):
  author_dict[id] = authors
author_dict

def authors(docno):
  
    raw_authors = author_dict[docno]
    if isinstance(raw_authors, str):
      authors = raw_authors.split(';')
      num_authors = len(authors)
      return num_authors

    return 1

def _features(row):
    f1 = authors(row["docno"])
    features = np.append(row['features'], np.array([f1]))
    return features

fbr = pt.FeaturesBatchRetrieve(index, controls = {"wmodel": "BM25"}, features=["WMODEL:TF_IDF", "WMODEL:PL2"]) 
#br = pt.BatchRetrieve(index, wmodel="BM25") >> pt.apply.doc_features(_features)

p = fbr >> pt.apply.doc_features(_features)
#p = br >> pt.apply.doc_features(_features)

p.transform("coronavirus origin")

/Users/nicolasrehbach/opt/anaconda3/lib/python3.9/site-packages/pyterrier/ops.py:331: FutureWarning: .transform() should be passed a dataframe. Use .search() to execute a single query.
  topics = m.transform(topics)


,qid,query,docid,rank,features,docno,score
0,1,coronavirus origin,122804,0,"[4.54079177445082, 3.834164304234873, 6.0]",75773gwg,11.578681
1,1,coronavirus origin,122805,1,"[4.026248144452075, 3.7774231171461947, 6.0]",kn2z7lho,11.578681
2,1,coronavirus origin,122806,2,"[4.028116303793698, 3.7583516835117754, 6.0]",4fb291hq,11.578681
3,1,coronavirus origin,135326,3,"[4.028116303793698, 3.7583516835117754, 3.0]",ne5r4d4b,11.452880
4,1,coronavirus origin,187888,4,"[4.293606847547854, 3.5355264383319778, 6.0]",hl967ekh,11.428047
...,...,...,...,...,...,...,...
995,1,coronavirus origin,186498,995,"[5.533836763230438, 5.025852140277717, 3.0]",w8cvq0m5,7.256041
996,1,coronavirus origin,10395,996,"[5.643668013169851, 4.872270617122758, 7.0]",l8a7lzhb,7.256040
997,1,coronavirus origin,134989,997,"[4.339779552413013, 4.1812942635313695, 12.0]",nh06fp82,7.256040
998,1,coronavirus origin,73423,998,"[5.164322402539175, 4.38705805069448, 3.0]",i758v1vb,7.252675


In [20]:
lmart_x_pipe = fbr >> pt.apply.doc_features(_features) >> pt.ltr.apply_learned_model(lmart_x, form="ltr")
lmart_x_pipe.fit(train_topics, train_qrels, validation_topics, validation_qrels)

[16:58:54] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/learner.cc:767: 
Parameters: { "verbose" } are not used.



In [21]:
results = pt.Experiment([PL2, lmart_x_pipe], test_topics, qrels, ["map"], names=["PL2 (Baseline)",  "LMART"])


In [22]:
results

,name,map
0,PL2 (Baseline),0.313517
1,LMART,0.146289


# How well do the algorithm work without any features?

Following, the five testes ranking algorithms will be put to the test. We are using the PL2 algorithm as a baseline and compare it to LambdaMart, LightGBM, Random Forest, FastRank with Gradient Ascent and FastRank with Random Forest.

In [23]:
#fbr = pt.BatchRetrieve(index, wmodel="BM25")
pl2 = pt.BatchRetrieve(index, wmodel="PL2")

fbr = pt.FeaturesBatchRetrieve(index, controls = {"wmodel": "BM25"}, features=["WMODEL:BM25"])

#p = fbr >> pt.apply.doc_features(_features)
# p = br >> pt.apply.doc_features(_features)

In [24]:
lmart_x = xgb.sklearn.XGBRanker(objective='rank:ndcg',
      learning_rate=0.1,
      gamma=1.0,
      min_child_weight=0.1,
      max_depth=6,
      verbose=2,
      random_state=42,
      num_round =  10)

In [98]:
import lightgbm as lgb
# this configures LightGBM as LambdaMART
lmart_l = lgb.LGBMRanker(task="train",
    min_data_in_leaf=1,
    min_sum_hessian_in_leaf=100,
    max_bin=255,
    num_leaves=7,
    objective="lambdarank",
    metric="ndcg",
    ndcg_eval_at=[1, 3, 5, 10],
    learning_rate= .1,
    importance_type="gain",
    num_iterations=10)

In [26]:
train_request = fastrank.TrainRequest.coordinate_ascent()
params = train_request.params
train_request.measure = 'ndcg'
params.init_random = True
params.normalize = True
params.seed = 1234567

In [27]:
train_request_forest = fastrank.TrainRequest.random_forest()
params = train_request_forest.params
train_request_forest.measure = 'ndcg'
params.num_trees = 5
params.feature_sampling_rate = 0.5
params.instance_sampling_rate = 0.5
params.seed = 1234567

In [28]:
rf = RandomForestRegressor(n_estimators= 300)
rf_pipe = fbr >> pt.ltr.apply_learned_model(rf)
rf_pipe.fit(train_topics, qrels)

In [29]:
lmart_x_pipe = fbr >> pt.ltr.apply_learned_model(lmart_x, form="ltr")
lmart_x_pipe.fit(train_topics, train_qrels, validation_topics, validation_qrels)

lmart_l_pipe = fbr >> pt.ltr.apply_learned_model(lmart_l, form="ltr")
lmart_l_pipe.fit(train_topics, train_qrels, validation_topics, validation_qrels)

rf_pipe = fbr >> pt.ltr.apply_learned_model(rf)
rf_pipe.fit(train_topics, qrels)

ca_pipe = fbr >> pt.ltr.apply_learned_model(train_request, form="fastrank")
ca_pipe.fit(train_topics, qrels)

fr_rf_pipe = fbr >> pt.ltr.apply_learned_model(train_request_forest, form="fastrank")
fr_rf_pipe.fit(train_topics, qrels)

[17:00:25] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/learner.cc:767: 
Parameters: { "num_round", "verbose" } are not used.



/Users/nicolasrehbach/opt/anaconda3/lib/python3.9/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Warning] min_data_in_leaf is set=1, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=1
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=100, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=100
[1]	valid_0's ndcg@1: 0.133333	valid_0's ndcg@2: 0.133333	valid_0's ndcg@3: 0.148976	valid_0's ndcg@4: 0.157555	valid_0's ndcg@5: 0.150003
[2]	valid_0's ndcg@1: 0.133333	valid_0's ndcg@2: 0.133333	valid_0's ndcg@3: 0.148976	valid_0's ndcg@4: 0.157555	valid_0's ndcg@5: 0.150003
[3]	valid_0's ndcg@1: 0.133333	valid_0's ndcg@2: 0.133333	valid_0's ndcg@3: 0.148976	valid_0's ndcg@4: 0.157555	valid_0's ndcg@5: 0.150003
[4]	valid_0's ndcg@1: 0.133333	valid_0's ndcg@2: 0.133333	valid_0's ndcg@3: 0.148976	valid_0's ndcg@4: 0.157555	valid_0's ndcg@5: 0.150003
[5]	valid_0's ndcg@1: 0.133333	valid_0's ndcg@2: 0.133333	valid_0's ndcg@3: 0.148976	valid_0's ndcg@4: 0.157555	valid_0's ndcg@5: 0.150003
[6]	valid_0's ndcg@1: 0.133333	valid_0's ndcg

In [30]:
results = pt.Experiment([PL2, lmart_x_pipe, lmart_l_pipe, rf_pipe, ca_pipe, fr_rf_pipe], test_topics, qrels, ["ndcg", "map", "recip_rank"], 
                        names=["PL2 (Baseline)", "LMART", "LMART LightGBM", "Random Forest", "Fastrank Coordinate Ascend", "Fastrank Random Forest"], baseline = 0, filter_by_topics = True)

In [31]:
results

,name,map,recip_rank,ndcg,map +,map -,map p-value,recip_rank +,recip_rank -,recip_rank p-value,ndcg +,ndcg -,ndcg p-value
0,PL2 (Baseline),0.313517,0.909091,0.523699,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,LMART,0.166378,0.406146,0.436964,0.0,10.0,0.005058,0.0,8.0,0.001764,1.0,9.0,0.004475
2,LMART LightGBM,0.132631,0.480101,0.421087,0.0,10.0,0.004582,0.0,7.0,0.006520,1.0,9.0,0.003567
3,Random Forest,0.147864,0.367699,0.422179,0.0,10.0,0.003030,0.0,8.0,0.002004,1.0,9.0,0.002634
4,Fastrank Coordinate Ascend,0.283973,0.914286,0.519278,1.0,9.0,0.042773,1.0,0.0,0.343436,2.0,8.0,0.471011
5,Fastrank Random Forest,0.153793,0.463393,0.426363,0.0,10.0,0.001262,1.0,7.0,0.011659,1.0,9.0,0.001116


**Summarizing**:
Based on three measures (MAP, Recip Rank and the ndcg) we can observe that PL2 outperformed all models except the Fastrank Coordinate Ascend. The models are currently just a reranking the BM25 score.

## Applying recency boost

In [35]:
metadata['publish_year'] = pd.to_datetime(metadata['publish_time'])
metadata['publish_year'] = metadata['publish_year'].dt.strftime('%Y')
metadata['publish_year'] = metadata['publish_year'].fillna(0)
metadata['publish_year'] = metadata['publish_year'].astype(str).astype(int)

metadata['publish_year_feature'] = 0

metadata.loc[metadata['publish_year'] < 2019, 'publish_year_feature'] = 1
metadata.loc[metadata['publish_year'] == 2019, 'publish_year_feature'] = 2
metadata.loc[metadata['publish_year'] >= 2020, 'publish_year_feature'] = 3


In [36]:
date_dict = {}

for id, dates in zip(metadata['cord_uid'], metadata['publish_year_feature']):
    date_dict[id] = dates


In [37]:
def dates(docno):
    raw_dates = date_dict[docno]
    return raw_dates

def _features(row):
    f1 = dates(row["docno"])
    features = np.append(row['features'], np.array([f1]))
    return features

In [38]:
p = fbr >> pt.apply.doc_features(_features)

In [39]:
p.transform("coronavirus origin")

/Users/nicolasrehbach/opt/anaconda3/lib/python3.9/site-packages/pyterrier/ops.py:331: FutureWarning: .transform() should be passed a dataframe. Use .search() to execute a single query.
  topics = m.transform(topics)


,qid,query,docid,rank,features,docno,score
0,1,coronavirus origin,122804,0,"[7.243034202814337, 3.0]",75773gwg,11.578681
1,1,coronavirus origin,122805,1,"[7.252674838560409, 3.0]",kn2z7lho,11.578681
2,1,coronavirus origin,122806,2,"[7.256040044023486, 3.0]",4fb291hq,11.578681
3,1,coronavirus origin,135326,3,"[7.256040044023486, 1.0]",ne5r4d4b,11.452880
4,1,coronavirus origin,187888,4,"[7.267153309588337, 3.0]",hl967ekh,11.428047
...,...,...,...,...,...,...,...
995,1,coronavirus origin,186498,995,"[8.965273511062476, 3.0]",w8cvq0m5,7.256041
996,1,coronavirus origin,10395,996,"[9.358174003925521, 1.0]",l8a7lzhb,7.256040
997,1,coronavirus origin,134989,997,"[7.81745407521776, 1.0]",nh06fp82,7.256040
998,1,coronavirus origin,73423,998,"[8.711893939558887, 3.0]",i758v1vb,7.252675


In [40]:
lmart_x_pipe = fbr >> pt.apply.doc_features(_features) >> pt.ltr.apply_learned_model(lmart_x, form="ltr")
lmart_x_pipe.fit(train_topics, train_qrels, validation_topics, validation_qrels)

lmart_l_pipe = fbr >> pt.apply.doc_features(_features) >> pt.ltr.apply_learned_model(lmart_l, form="ltr")
lmart_l_pipe.fit(train_topics, train_qrels, validation_topics, validation_qrels)

rf_pipe = fbr >> pt.apply.doc_features(_features) >> pt.ltr.apply_learned_model(rf)
rf_pipe.fit(train_topics, qrels)

ca_pipe = fbr >> pt.apply.doc_features(_features) >> pt.ltr.apply_learned_model(train_request, form="fastrank")
ca_pipe.fit(train_topics, train_qrels)

fr_rf_pipe = fbr >> pt.apply.doc_features(_features) >> pt.ltr.apply_learned_model(train_request_forest, form="fastrank")
fr_rf_pipe.fit(train_topics, train_qrels)

[17:04:23] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/learner.cc:767: 
Parameters: { "num_round", "verbose" } are not used.



/Users/nicolasrehbach/opt/anaconda3/lib/python3.9/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Warning] min_data_in_leaf is set=1, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=1
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=100, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=100
[1]	valid_0's ndcg@1: 0.133333	valid_0's ndcg@2: 0.133333	valid_0's ndcg@3: 0.148976	valid_0's ndcg@4: 0.157555	valid_0's ndcg@5: 0.150003
[2]	valid_0's ndcg@1: 0.133333	valid_0's ndcg@2: 0.133333	valid_0's ndcg@3: 0.148976	valid_0's ndcg@4: 0.157555	valid_0's ndcg@5: 0.150003
[3]	valid_0's ndcg@1: 0.133333	valid_0's ndcg@2: 0.133333	valid_0's ndcg@3: 0.148976	valid_0's ndcg@4: 0.157555	valid_0's ndcg@5: 0.150003
[4]	valid_0's ndcg@1: 0.133333	valid_0's ndcg@2: 0.133333	valid_0's ndcg@3: 0.148976	valid_0's ndcg@4: 0.157555	valid_0's ndcg@5: 0.150003
[5]	valid_0's ndcg@1: 0.133333	valid_0's ndcg@2: 0.133333	valid_0's ndcg@3: 0.148976	valid_0's ndcg@4: 0.157555	valid_0's ndcg@5: 0.150003
[6]	valid_0's ndcg@1: 0.133333	valid_0's ndcg

In [41]:
results = pt.Experiment([PL2, lmart_x_pipe, lmart_l_pipe, rf_pipe, ca_pipe, fr_rf_pipe], test_topics, qrels, ["ndcg", "map", "recip_rank"], 
                        names=["PL2 (Baseline)", "LMART", "LMART LightGBM", "Random Forest", "Fastrank Coordinate Ascend", "Fastrank Random Forest"], baseline = 0, filter_by_topics = True )

In [42]:
results

,name,map,recip_rank,ndcg,map +,map -,map p-value,recip_rank +,recip_rank -,recip_rank p-value,ndcg +,ndcg -,ndcg p-value
0,PL2 (Baseline),0.313517,0.909091,0.523699,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,LMART,0.168598,0.450476,0.445947,2.0,8.0,0.012282,0.0,8.0,0.002137,2.0,8.0,0.014904
2,LMART LightGBM,0.132631,0.480101,0.421087,0.0,10.0,0.004582,0.0,7.0,0.006520,1.0,9.0,0.003567
3,Random Forest,0.176554,0.482423,0.445811,0.0,10.0,0.006529,0.0,7.0,0.007500,1.0,9.0,0.010993
4,Fastrank Coordinate Ascend,0.321934,0.950000,0.539068,5.0,5.0,0.456311,1.0,0.0,0.343436,6.0,4.0,0.068947
5,Fastrank Random Forest,0.191845,0.478667,0.452389,1.0,9.0,0.006928,0.0,8.0,0.002206,1.0,9.0,0.005969


In [43]:
topic = pd.DataFrame(fbr.transform("coronavirus origin"))

/var/folders/vc/33g3tqwn4c59qnzswhnf6ry00000gn/T/ipykernel_3003/1828645794.py:1: FutureWarning: .transform() should be passed a dataframe. Use .search() to execute a single query.
  topic = pd.DataFrame(fbr.transform("coronavirus origin"))


In [44]:
results = pt.Experiment([PL2, ca_pipe], topic, qrels, ["ndcg", "map", "recip_rank"], 
                        names=["PL2 (Baseline)", "Fastrank Coordinate Ascend"],filter_by_topics = True, baseline = 0)

/Users/nicolasrehbach/opt/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3702: RuntimeWarning: Degrees of freedom <= 0 for slice
  return _methods._var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/nicolasrehbach/opt/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:253: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [45]:
results

,name,map,recip_rank,ndcg,map +,map -,map p-value,recip_rank +,recip_rank -,recip_rank p-value,ndcg +,ndcg -,ndcg p-value
0,PL2 (Baseline),0.151897,1.0,0.362539,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Fastrank Coordinate Ascend,0.091757,0.5,0.316739,0.0,1.0,NaN,0.0,1.0,NaN,0.0,1.0,NaN


## Applying Journal impact

In [46]:
journal_impact = pd.read_csv('./data/scimagojr 2020.csv', sep = ';', header = None)
journal_abbrev = pd.read_csv('./data/wos_abbrev_table.csv', sep = ';', header = None)
journal_impact=journal_impact.T.set_index(0).T
journal_abbrev=journal_abbrev.T.set_index(0).T
journal_abbrev = journal_abbrev[['full', 'abbrev']]

/Users/nicolasrehbach/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (0,1,5,7,8,9,10,11,12) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [47]:
journal_impact = journal_impact[['Title', 'SJR', 'H index']]

In [48]:
journals = metadata[['cord_uid','journal']]
journals = journals.drop_duplicates()
journals = pd.DataFrame(journals)

In [49]:
journals['journal'] = journals['journal'].str.lower()
journal_impact['Title'] = journal_impact['Title'].str.lower()
journal_abbrev['full'] = journal_abbrev['full'].str.lower()
journal_abbrev['abbrev'] = journal_abbrev['abbrev'].str.lower()

In [50]:
journals = pd.merge(journals, journal_abbrev, left_on='journal', right_on='abbrev', how='left')

In [51]:
journals = journals[['cord_uid', 'journal', 'full', 'abbrev']]
journals['full'] = journals['full'].fillna(journals['journal'])
journals['abbrev'] = journals['abbrev'].fillna(journals['journal'])

In [52]:
journals

,cord_uid,journal,full,abbrev
0,ug7v899j,bmc infect dis,bmc infectious diseases,bmc infect dis
1,02tnwd4m,respir res,respiratory research,respir res
2,ejv2xln0,respir res,respiratory research,respir res
3,2b73a28n,respir res,respiratory research,respir res
4,9785vg6d,respir res,respiratory research,respir res
...,...,...,...,...
217298,z4ro6lmh,infection,infection,infection
217299,hi8k8wvb,physica b: condensed matter,physica b: condensed matter,physica b: condensed matter
217300,ma3ndg41,catheter cardiovasc interv,catheter cardiovasc interv,catheter cardiovasc interv
217301,wh10285j,ann surg,annals of surgery,ann surg


In [53]:
journals['cord_uid'] = journals['cord_uid'].drop_duplicates()

In [54]:
metadata_journals = pd.merge(metadata, journals , left_on='cord_uid', right_on='cord_uid', how='left')

In [55]:
metadata_journals = metadata_journals.merge(journal_impact, how = 'left', left_on = "full", right_on = "Title")

In [56]:
#merged = metadata.merge(journal_impact, how = 'left', left_on = "journal", right_on = "Title")
metadata_journals['H index'] = [float(str(i).replace(",", "")) for i in metadata_journals['H index']]
#merged['H index'] = merged['H index'].fillna(merged['H index'].mean())
#merged['norm_H_index']=(merged['H index']-merged['H index'].min())/(merged['H index'].max()-merged['H index'].min())

In [57]:
metadata_journals['H index'] = [float(str(i).replace(",", "")) for i in metadata_journals['H index']]
metadata_journals = metadata_journals.rename(columns={"H index": "H_index"})

In [58]:
metadata_journals['SJR'] = [float(str(i).replace(",", "")) for i in metadata_journals['SJR']]


In [59]:
metadata_journals['H_index'].describe()

count    88969.000000
mean       170.107476
std        189.732422
min          0.000000
25%         70.000000
50%        116.000000
75%        205.000000
max       1276.000000
Name: H_index, dtype: float64

In [60]:
lower = metadata_journals['H_index'].describe()[4]
middle = metadata_journals['H_index'].describe()[5]
upper = metadata_journals['H_index'].describe()[6]

In [61]:
lower, middle, upper

(70.0, 116.0, 205.0)

In [62]:
metadata_journals['H_index'].fillna(value = lower, inplace = True)

metadata_journals['H_index'].loc[metadata_journals['H_index'] < lower] = 1
metadata_journals['H_index'].loc[metadata_journals['H_index'] == lower] = 1.5
metadata_journals.loc[metadata_journals['H_index'].between(lower, middle), 'H_index'] = 2.0
metadata_journals.loc[metadata_journals['H_index'].between(middle,upper), 'H_index'] = 2.5
metadata_journals.loc[metadata_journals['H_index'].between(upper, 1276), 'H_index'] = 3.5

/Users/nicolasrehbach/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [63]:
lower = metadata_journals['SJR'].describe()[4]
middle = metadata_journals['SJR'].describe()[5]
upper = metadata_journals['SJR'].describe()[6]

In [64]:
lower, middle, upper

(751.0, 1175.0, 2134.0)

In [65]:
metadata_journals['SJR'].fillna(value = lower, inplace = True)

metadata_journals['SJR'].loc[metadata_journals['SJR'] < lower] = 1
metadata_journals['SJR'].loc[metadata_journals['SJR'] == lower] = 1.5
metadata_journals.loc[metadata_journals['SJR'].between(lower, middle), 'SJR'] = 2.0
metadata_journals.loc[metadata_journals['SJR'].between(middle,upper), 'SJR'] = 2.5
metadata_journals.loc[metadata_journals['SJR'].between(upper, 37461), 'SJR'] = 3.5

In [66]:
metadata_journals['H_index'].describe()

count    192847.000000
mean          1.846661
std           0.715208
min           1.000000
25%           1.500000
50%           1.500000
75%           2.000000
max           3.500000
Name: H_index, dtype: float64

In [67]:
metadata_journals['SJR'].describe()

count    192847.000000
mean          1.842694
std           0.714493
min           1.000000
25%           1.500000
50%           1.500000
75%           2.000000
max           3.500000
Name: SJR, dtype: float64

In [68]:
sjr_dict = {}
h_dict = {}

for id, sjr in zip(metadata_journals['cord_uid'], metadata_journals['SJR']):
    sjr_dict[id] = sjr

for id, h_id in zip(metadata_journals['cord_uid'], metadata_journals['H_index']):
    h_dict[id] = h_id


In [69]:
def dates(docno):
    raw_dates = date_dict[docno]
    return raw_dates

def sjr(docno):
    raw_sjr = sjr_dict[docno]
    return raw_sjr

def h_idx(docno):
    raw_h_idx = h_dict[docno]
    return raw_h_idx

def _features(row):
    f1 = dates(row["docno"])
    f2 = sjr(row["docno"])
    f3 = h_idx(row['docno'])
    features = np.append(row['features'], np.array([f1, f2,  f3]))
    #features = np.append(row['features'], np.array([f1, f2, f3]))
#    features = np.append(row['features'], np.array([f2]))
    return features

In [70]:
fbr = pt.FeaturesBatchRetrieve(index, controls = {"wmodel": "BM25"}, features=["WMODEL:BM25"]) 
p = fbr >> pt.apply.doc_features(_features)
p.transform("coronavirus origin")

/Users/nicolasrehbach/opt/anaconda3/lib/python3.9/site-packages/pyterrier/ops.py:331: FutureWarning: .transform() should be passed a dataframe. Use .search() to execute a single query.
  topics = m.transform(topics)


,qid,query,docid,rank,features,docno,score
0,1,coronavirus origin,122804,0,"[7.243034202814337, 3.0, 1.5, 1.5]",75773gwg,11.578681
1,1,coronavirus origin,122805,1,"[7.252674838560409, 3.0, 1.5, 1.5]",kn2z7lho,11.578681
2,1,coronavirus origin,122806,2,"[7.256040044023486, 3.0, 1.5, 1.5]",4fb291hq,11.578681
3,1,coronavirus origin,135326,3,"[7.256040044023486, 1.0, 3.5, 3.5]",ne5r4d4b,11.452880
4,1,coronavirus origin,187888,4,"[7.267153309588337, 3.0, 1.5, 1.5]",hl967ekh,11.428047
...,...,...,...,...,...,...,...
995,1,coronavirus origin,186498,995,"[8.965273511062476, 3.0, 2.0, 2.0]",w8cvq0m5,7.256041
996,1,coronavirus origin,10395,996,"[9.358174003925521, 1.0, 2.5, 1.5]",l8a7lzhb,7.256040
997,1,coronavirus origin,134989,997,"[7.81745407521776, 1.0, 2.0, 2.5]",nh06fp82,7.256040
998,1,coronavirus origin,73423,998,"[8.711893939558887, 3.0, 1.5, 1.5]",i758v1vb,7.252675


In [71]:
lmart_x_pipe = fbr >> pt.apply.doc_features(_features) >> pt.ltr.apply_learned_model(lmart_x, form="ltr")
lmart_x_pipe.fit(train_topics, train_qrels, validation_topics, validation_qrels)

lmart_l_pipe = fbr >> pt.apply.doc_features(_features) >> pt.ltr.apply_learned_model(lmart_l, form="ltr")
lmart_l_pipe.fit(train_topics, train_qrels, validation_topics, validation_qrels)

rf_pipe = fbr >> pt.apply.doc_features(_features) >> pt.ltr.apply_learned_model(rf)
rf_pipe.fit(train_topics, qrels)

ca_pipe = fbr >> pt.apply.doc_features(_features) >> pt.ltr.apply_learned_model(train_request, form="fastrank")
ca_pipe.fit(train_topics, qrels)

fr_rf_pipe = fbr >> pt.apply.doc_features(_features) >> pt.ltr.apply_learned_model(train_request_forest, form="fastrank")
fr_rf_pipe.fit(train_topics, qrels)

[17:08:27] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/learner.cc:767: 
Parameters: { "num_round", "verbose" } are not used.



/Users/nicolasrehbach/opt/anaconda3/lib/python3.9/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Warning] min_data_in_leaf is set=1, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=1
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=100, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=100
[1]	valid_0's ndcg@1: 0.133333	valid_0's ndcg@2: 0.133333	valid_0's ndcg@3: 0.148976	valid_0's ndcg@4: 0.157555	valid_0's ndcg@5: 0.150003
[2]	valid_0's ndcg@1: 0.133333	valid_0's ndcg@2: 0.133333	valid_0's ndcg@3: 0.148976	valid_0's ndcg@4: 0.157555	valid_0's ndcg@5: 0.150003
[3]	valid_0's ndcg@1: 0.133333	valid_0's ndcg@2: 0.133333	valid_0's ndcg@3: 0.148976	valid_0's ndcg@4: 0.157555	valid_0's ndcg@5: 0.150003
[4]	valid_0's ndcg@1: 0.133333	valid_0's ndcg@2: 0.133333	valid_0's ndcg@3: 0.148976	valid_0's ndcg@4: 0.157555	valid_0's ndcg@5: 0.150003
[5]	valid_0's ndcg@1: 0.133333	valid_0's ndcg@2: 0.133333	valid_0's ndcg@3: 0.148976	valid_0's ndcg@4: 0.157555	valid_0's ndcg@5: 0.150003
[6]	valid_0's ndcg@1: 0.133333	valid_0's ndcg

In [72]:
results = pt.Experiment([PL2, lmart_x_pipe, lmart_l_pipe, rf_pipe, ca_pipe, fr_rf_pipe], test_topics, qrels, ["ndcg", "map", "recip_rank"], 
                        names=["PL2 (Baseline)", "LMART", "LMART LightGBM", "Random Forest", "Fastrank Coordinate Ascend", "Fastrank Random Forest"], baseline = 0, filter_by_topics = True )


In [73]:
results

,name,map,recip_rank,ndcg,map +,map -,map p-value,recip_rank +,recip_rank -,recip_rank p-value,ndcg +,ndcg -,ndcg p-value
0,PL2 (Baseline),0.313517,0.909091,0.523699,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,LMART,0.168598,0.450476,0.445947,2.0,8.0,0.012282,0.0,8.0,0.002137,2.0,8.0,0.014904
2,LMART LightGBM,0.132631,0.480101,0.421087,0.0,10.0,0.004582,0.0,7.0,0.006520,1.0,9.0,0.003567
3,Random Forest,0.180172,0.368056,0.446702,0.0,10.0,0.006925,1.0,8.0,0.001834,1.0,9.0,0.007983
4,Fastrank Coordinate Ascend,0.321001,0.883333,0.536345,5.0,5.0,0.469170,1.0,1.0,0.667139,6.0,4.0,0.073502
5,Fastrank Random Forest,0.129692,0.383410,0.403252,0.0,10.0,0.003957,1.0,6.0,0.006977,1.0,9.0,0.001453


The SJR and H index helped Fastrank however not LMAR or RF. Generally, about 88000 columns have an associated journal ranking now. The others are 

## Citation rank

# Applying the topic cites to all qrels

In [74]:
def get_topic_cites2(input_df):
    df = input_df
    # merge the fbr (pooled documents) with our metadata and remove duplicates to minimize API calls
    df = input_df.merge(metadata, left_on = "docno", right_on = "cord_uid", how = "left") 
    df = df[['qid', 'docno', 'label', 'iteration', 'cord_uid', 'title', 'doi',
                    'abstract', 'publish_time', 'authors', 'journal']]
    #fbr_meta = fbr_meta.drop_duplicates(subset='doi', keep="first")
    #fbr_meta = fbr_meta.drop_duplicates(subset='title', keep="first")
    df = df.reset_index()
    df = df.drop(columns = 'index')
    #fbr_meta.update('"' + fbr_meta[['title']].astype(str) + '"')
    df = df.drop_duplicates(subset='title', keep="first")

    cites_df = pd.DataFrame()
    for i in range(len(df)):
        try:
            title_information = requests.get(
                'https://api.openalex.org/works?filter=title.search:'+df['title'][i]
                ).json()['results'][0]
            current_title = pd.DataFrame.from_dict(title_information, orient='index')
            current_title = current_title.transpose()
            cites_df = cites_df.append(current_title)
            print("Request number:", i)
        # in case DOI and title is missing, no information is printed
        except:
            try:
                doi_information = requests.get(
                    'https://api.openalex.org/works?filter=doi:https://doi.org/'+df['doi'][i]
                    ).json()['results'][0]
                    #print('Retrieved:',fbr_meta['doi'][i])
                    # append the current document information to our main dataframe
                current_doi = pd.DataFrame.from_dict(doi_information, orient='index')
                current_doi = current_doi.transpose()
                cites_df = cites_df.append(current_doi)
                print("Request number:", i)
            except:
                print("No title or doi information on OpenAlex found")
    # finally, we are merging our OpenAlex data with our fbr and meta data by merging over the doi
    cites_df = cites_df[['doi', 'title', 'referenced_works', 'related_works', 'cited_by_count', 'counts_by_year']]
    cites_df = cites_df.reset_index()
    cites_df = cites_df.drop(columns = 'index')
    cites_df['doi'] = cites_df.doi.str.replace('https://doi.org/', '')
    #fbr_meta_cites = fbr_meta.merge(cites_df, left_on = 'title', right_on = 'title')
    return cites_df

In [75]:
qrels['docno'].drop_duplicates(keep="first")

0        005b2j4b
1        00fmeepz
2        010vptx3
3        0194oljo
4        021q9884
           ...   
69305    zn10rnrm
69308    zstmdt4n
69310    zth8ffy3
69312    zv4nbz9p
69313    zvop8bxh
Name: docno, Length: 37924, dtype: object

In [76]:
cites_df1 = pd.read_csv('../WIR-Project/cites_df1.csv')
cites_df2 = pd.read_csv('../WIR-Project/cites_df2.csv')
cites_df3 = pd.read_csv('../WIR-Project/cites_df3.csv')
cites_df4 = pd.read_csv('../WIR-Project/cites_df4.csv')

cites_df = cites_df1.append(cites_df2)
cites_df = cites_df.append(cites_df3)
cites_df = cites_df.append(cites_df4)
qrels = dataset.get_qrels()

In [77]:
cites_df['title'] = cites_df['title'].str.lower()

meta = metadata
meta['title'] = meta['title'].str.lower()

meta_cites = cites_df.merge(meta, left_on = 'title', right_on = 'title', how = 'right')
meta_cites = meta_cites[['cited_by_count', 'cord_uid']]
meta_cites = meta_cites.rename(columns={"cord_uid": "docno"})

In [78]:
qrels = qrels.reset_index()
qrels = qrels.rename(columns={"index":"indexer"})

In [79]:
qrels

,indexer,qid,docno,label,iteration
0,0,1,005b2j4b,2,4.5
1,1,1,00fmeepz,1,4
2,2,1,010vptx3,2,0.5
3,3,1,0194oljo,1,2.5
4,4,1,021q9884,1,4
...,...,...,...,...,...
69313,69313,50,zvop8bxh,2,5
69314,69314,50,zwf26o63,1,5
69315,69315,50,zwsvlnwe,0,5
69316,69316,50,zxr01yln,1,5


In [80]:
meta_cites = meta_cites.drop_duplicates()
merged_df = pd.merge(qrels, meta_cites, on='docno', how='left', indicator=True)

In [81]:
merged_df = merged_df.drop_duplicates(subset='indexer', keep="first")
merged_df = merged_df.reset_index()
merged_df = merged_df.drop(columns = 'index')

In [82]:
merged_df

,indexer,qid,docno,label,iteration,cited_by_count,_merge
0,0,1,005b2j4b,2,4.5,8.0,both
1,1,1,00fmeepz,1,4,NaN,both
2,2,1,010vptx3,2,0.5,917.0,both
3,3,1,0194oljo,1,2.5,31.0,both
4,4,1,021q9884,1,4,0.0,both
...,...,...,...,...,...,...,...
69313,69313,50,zvop8bxh,2,5,NaN,both
69314,69314,50,zwf26o63,1,5,0.0,both
69315,69315,50,zwsvlnwe,0,5,12.0,both
69316,69316,50,zxr01yln,1,5,33.0,both


In [83]:
qrels = qrels.drop(columns=['indexer'])
result = qrels.eq(merged_df[['qid', 'docno', 'label', 'iteration']], axis='index')
result = result.all(axis=1)

In [84]:
a = merged_df[['qid', 'docno', 'label', 'iteration']]
b = qrels

if a.equals(b):
    print("The DataFrames are identical.")
else:
    print("The DataFrames are not identical.")

The DataFrames are identical.


In [85]:
merged_df['cited_by_count'].describe()

count    48306.000000
mean       169.759926
std        757.097184
min          0.000000
25%         11.000000
50%         35.000000
75%        111.000000
max      30662.000000
Name: cited_by_count, dtype: float64

In [86]:
lower = merged_df['cited_by_count'].describe()[4]
middle = merged_df['cited_by_count'].describe()[5]
upper = merged_df['cited_by_count'].describe()[6]

merged_df['cited_by_count'] = merged_df['cited_by_count'].fillna(value=lower)
merged_df['cited_by_count'].loc[merged_df['cited_by_count'] < lower] = 1
merged_df.loc[merged_df['cited_by_count'].between(lower, middle), 'cited_by_count'] = 2
merged_df.loc[merged_df['cited_by_count'].between(middle, upper), 'cited_by_count'] = 3
merged_df['cited_by_count'].loc[merged_df['cited_by_count'] > upper] = 4

/Users/nicolasrehbach/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [87]:
meta

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,...,journal,mag_id,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id,publish_year,publish_year_feature
0,ug7v899j,d1aafb70c066a2068b02786f8929fd9c900897fb,PMC,clinical features of culture-proven mycoplasma...,10.1186/1471-2334-1-6,PMC35282,11472636.0,no-cc,OBJECTIVE: This retrospective chart review des...,2001-07-04,...,BMC Infect Dis,NaN,NaN,NaN,document_parses/pdf_json/d1aafb70c066a2068b027...,document_parses/pmc_json/PMC35282.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...,NaN,2001,1
1,02tnwd4m,6b0567729c2143a66d737eb0a2f63f2dce2e5a7d,PMC,nitric oxide: a pro-inflammatory mediator in l...,10.1186/rr14,PMC59543,11667967.0,no-cc,Inflammatory diseases of the respiratory tract...,2000-08-15,...,Respir Res,NaN,NaN,NaN,document_parses/pdf_json/6b0567729c2143a66d737...,document_parses/pmc_json/PMC59543.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN,2000,1
2,ejv2xln0,06ced00a5fc04215949aa72528f2eeaae1d58927,PMC,surfactant protein-d and pulmonary host defense,10.1186/rr19,PMC59549,11667972.0,no-cc,Surfactant protein-D (SP-D) participates in th...,2000-08-25,...,Respir Res,NaN,NaN,NaN,document_parses/pdf_json/06ced00a5fc04215949aa...,document_parses/pmc_json/PMC59549.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN,2000,1
3,2b73a28n,348055649b6b8cf2b9a376498df9bf41f7123605,PMC,role of endothelin-1 in lung disease,10.1186/rr44,PMC59574,11686871.0,no-cc,Endothelin-1 (ET-1) is a 21 amino acid peptide...,2001-02-22,...,Respir Res,NaN,NaN,NaN,document_parses/pdf_json/348055649b6b8cf2b9a37...,document_parses/pmc_json/PMC59574.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN,2001,1
4,9785vg6d,5f48792a5fa08bed9f56016f4981ae2ca6031b32,PMC,gene expression in epithelial cells in respons...,10.1186/rr61,PMC59580,11686888.0,no-cc,Respiratory syncytial virus (RSV) and pneumoni...,2001-05-11,...,Respir Res,NaN,NaN,NaN,document_parses/pdf_json/5f48792a5fa08bed9f560...,document_parses/pmc_json/PMC59580.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN,2001,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192504,z4ro6lmh,203f36475be74229101548475d68352b939f8b5b,Medline; PMC,rapid radiological improvement of covid-19 pne...,10.1007/s15010-020-01449-w,PMC7299451,32557206,no-cc,NaN,2020-06-15,...,Infection,NaN,NaN,NaN,document_parses/pdf_json/203f36475be7422910154...,document_parses/pmc_json/PMC7299451.xml.json,https://doi.org/10.1007/s15010-020-01449-w; ht...,219729576.0,2020,3
192505,hi8k8wvb,9f1bc99798e8823e690697394dcb23533a45c60e,Elsevier; Medline; PMC,sars e protein in phospholipid bilayers: an an...,10.1016/j.physb.2004.11.015,PMC7127356,32288217,els-covid,Abstract We report on an anomalous X-ray refle...,2005-02-28,...,Physica B: Condensed Matter,NaN,NaN,NaN,document_parses/pdf_json/9f1bc99798e8823e69069...,document_parses/pmc_json/PMC7127356.xml.json,https://www.ncbi.nlm.nih.gov/pubmed/32288217/;...,122247693.0,2005,1
192506,ma3ndg41,ffba777376718ef2a0dd74a8eab90e2bfacd240f,Medline; PMC,italian society of interventional cardiology (...,10.1002/ccd.28888,PMC7228289,32223063,no-cc,COVID‐19 pandemic raised the issue to guarante...,2020-04-11,...,Catheter Cardiovasc Interv,NaN,NaN,NaN,document_parses/pdf_json/ffba777376718ef2a0dd7...,document_parses/pmc_json/PMC7228289.xml.json,https://www.ncbi.nlm.nih.gov/pubmed/32223063/;...,214715941.0,2020,3
192507,wh10285j,d521c5a2dcbd79a5be606fcf586b1e0448344172,Medline; PMC,"nimble, together: a training program's respons...",10.1097/sla.0000000000003994,PMC7224622,32355117,cc-by-nc-nd,NaN,2020-04-29,...,Ann Surg,NaN,NaN,NaN,document_parses/pdf_json/d521c5a2dcbd79a5be606...,document_parses/pmc_json/PMC7224622.xml.json,https://www.ncbi.nlm.nih.gov/pubmed/32355117/;...,218468770.0,2020,3


In [88]:
docno_cites = merged_df[['docno', 'cited_by_count']]
full_data = meta.merge(docno_cites, left_on= "cord_uid", right_on = "docno", how = "outer")
full_data['cited_by_count'] = full_data['cited_by_count'].fillna(value= 0)

In [91]:
citation_dic = {}

for id, cites in zip(full_data['cord_uid'], full_data['cited_by_count']):
    citation_dic[id] = cites

In [92]:
def dates(docno):
    raw_dates = date_dict[docno]
    return raw_dates

def sjr(docno):
    raw_sjr = sjr_dict[docno]
    return raw_sjr

def h_idx(docno):
    raw_h_idx = h_dict[docno]
    return raw_h_idx

def cites(docno):
    raw_cites = citation_dic[docno]
    return raw_cites

def _features(row):
    f1 = dates(row["docno"])
    f2 = sjr(row["docno"])
    f3 = h_idx(row["docno"])
    f4 = cites(row["docno"])
    features = np.append(row['features'], np.array([f1, f2, f4]))
    #features = np.append(row['features'], np.array([f1, f2, f3]))
#    features = np.append(row['features'], np.array([f2]))
    return features

In [93]:
qrels

,qid,docno,label,iteration
0,1,005b2j4b,2,4.5
1,1,00fmeepz,1,4
2,1,010vptx3,2,0.5
3,1,0194oljo,1,2.5
4,1,021q9884,1,4
...,...,...,...,...
69313,50,zvop8bxh,2,5
69314,50,zwf26o63,1,5
69315,50,zwsvlnwe,0,5
69316,50,zxr01yln,1,5


In [94]:
fbr = pt.FeaturesBatchRetrieve(index, controls = {"wmodel": "BM25"}, features=["WMODEL:BM25"]) 
#br = pt.BatchRetrieve(index, wmodel="BM25") >> pt.apply.doc_features(_features)

fbr >> pt.apply.doc_features(_features)
p = fbr >> pt.apply.doc_features(_features)

p.transform("coronavirus origin")

/Users/nicolasrehbach/opt/anaconda3/lib/python3.9/site-packages/pyterrier/ops.py:331: FutureWarning: .transform() should be passed a dataframe. Use .search() to execute a single query.
  topics = m.transform(topics)


,qid,query,docid,rank,features,docno,score
0,1,coronavirus origin,122804,0,"[7.243034202814337, 3.0, 1.5, 3.0]",75773gwg,11.578681
1,1,coronavirus origin,122805,1,"[7.252674838560409, 3.0, 1.5, 3.0]",kn2z7lho,11.578681
2,1,coronavirus origin,122806,2,"[7.256040044023486, 3.0, 1.5, 3.0]",4fb291hq,11.578681
3,1,coronavirus origin,135326,3,"[7.256040044023486, 1.0, 3.5, 4.0]",ne5r4d4b,11.452880
4,1,coronavirus origin,187888,4,"[7.267153309588337, 3.0, 1.5, 3.0]",hl967ekh,11.428047
...,...,...,...,...,...,...,...
995,1,coronavirus origin,186498,995,"[8.965273511062476, 3.0, 2.0, 4.0]",w8cvq0m5,7.256041
996,1,coronavirus origin,10395,996,"[9.358174003925521, 1.0, 2.5, 2.0]",l8a7lzhb,7.256040
997,1,coronavirus origin,134989,997,"[7.81745407521776, 1.0, 2.0, 0.0]",nh06fp82,7.256040
998,1,coronavirus origin,73423,998,"[8.711893939558887, 3.0, 1.5, 2.0]",i758v1vb,7.252675


In [95]:
lmart_x_pipe = fbr >> pt.apply.doc_features(_features) >> pt.ltr.apply_learned_model(lmart_x, form="ltr")
lmart_x_pipe.fit(train_topics, train_qrels, validation_topics, validation_qrels)

lmart_l_pipe = fbr >> pt.apply.doc_features(_features) >> pt.ltr.apply_learned_model(lmart_l, form="ltr")
lmart_l_pipe.fit(train_topics, train_qrels, validation_topics, validation_qrels)

rf_pipe = fbr >> pt.apply.doc_features(_features) >> pt.ltr.apply_learned_model(rf)
rf_pipe.fit(train_topics, qrels)

ca_pipe = fbr >> pt.apply.doc_features(_features) >> pt.ltr.apply_learned_model(train_request, form="fastrank")
ca_pipe.fit(train_topics, qrels)

fr_rf_pipe = fbr >> pt.apply.doc_features(_features) >> pt.ltr.apply_learned_model(train_request_forest, form="fastrank")
fr_rf_pipe.fit(train_topics, qrels)

[17:13:28] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/learner.cc:767: 
Parameters: { "num_round", "verbose" } are not used.



/Users/nicolasrehbach/opt/anaconda3/lib/python3.9/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Warning] min_data_in_leaf is set=1, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=1
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=100, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=100
[1]	valid_0's ndcg@1: 0.133333	valid_0's ndcg@2: 0.133333	valid_0's ndcg@3: 0.148976	valid_0's ndcg@4: 0.157555	valid_0's ndcg@5: 0.150003
[2]	valid_0's ndcg@1: 0.133333	valid_0's ndcg@2: 0.133333	valid_0's ndcg@3: 0.148976	valid_0's ndcg@4: 0.157555	valid_0's ndcg@5: 0.150003
[3]	valid_0's ndcg@1: 0.133333	valid_0's ndcg@2: 0.133333	valid_0's ndcg@3: 0.148976	valid_0's ndcg@4: 0.157555	valid_0's ndcg@5: 0.150003
[4]	valid_0's ndcg@1: 0.133333	valid_0's ndcg@2: 0.133333	valid_0's ndcg@3: 0.148976	valid_0's ndcg@4: 0.157555	valid_0's ndcg@5: 0.150003
[5]	valid_0's ndcg@1: 0.133333	valid_0's ndcg@2: 0.133333	valid_0's ndcg@3: 0.148976	valid_0's ndcg@4: 0.157555	valid_0's ndcg@5: 0.150003
[6]	valid_0's ndcg@1: 0.133333	valid_0's ndcg

In [96]:
results = pt.Experiment([PL2, lmart_x_pipe, lmart_l_pipe, rf_pipe, ca_pipe, fr_rf_pipe], test_topics, qrels, ["ndcg", "map", "recip_rank"], 
                        names=["PL2 (Baseline)", "LMART", "LMART LightGBM", "Random Forest", "Fastrank Coordinate Ascend", "Fastrank Random Forest"], baseline = 0, filter_by_topics = True )



In [97]:
results

,name,map,recip_rank,ndcg,map +,map -,map p-value,recip_rank +,recip_rank -,recip_rank p-value,ndcg +,ndcg -,ndcg p-value
0,PL2 (Baseline),0.313517,0.909091,0.523699,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,LMART,0.268962,0.562500,0.493576,5.0,5.0,0.293422,1.0,6.0,0.006738,3.0,7.0,0.203719
2,LMART LightGBM,0.132631,0.480101,0.421087,0.0,10.0,0.004582,0.0,7.0,0.006520,1.0,9.0,0.003567
3,Random Forest,0.283846,0.695000,0.503158,5.0,5.0,0.292949,1.0,4.0,0.121521,4.0,6.0,0.171682
4,Fastrank Coordinate Ascend,0.324026,1.000000,0.530322,5.0,5.0,0.654728,1.0,0.0,0.343436,6.0,4.0,0.627322
5,Fastrank Random Forest,0.222349,0.306930,0.453361,2.0,8.0,0.081753,1.0,8.0,0.002349,3.0,7.0,0.022688
